In [ ]:
# !nvidia-smi

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  with open(fn, 'wb') as f:
    f.write(uploaded[fn])
  print('File uploaded successfully!')

Saving presidents.txt to presidents.txt

User uploaded file "presidents.txt" with length 7979211 bytes

File uploaded successfully!


In [ ]:
!ls

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 838.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 40.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 61.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstall

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd

2024-07-04 09:04:25.698386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 09:04:25.698509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 09:04:25.870260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Read the plain text file
file_path = 'FILEPATH'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Convert to a pandas DataFrame
data = {'text': [line.strip() for line in lines]}
df = pd.DataFrame(data)

# Create a Dataset object
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 34368
})

In [ ]:
checkpoint = 'gpt2'

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize dataset
def tokenize_function(examples):
    tokens = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokens["labels"] = tokens["input_ids"].copy()  # Labels are the same as input_ids for language modeling
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/34368 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 34368
})

In [ ]:
# Load model for causal language modeling
model = AutoModelForCausalLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.177200
1000,0.145800
1500,0.139300
2000,0.134000
2500,0.129900
3000,0.124300
3500,0.123400
4000,0.122400


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=4296, training_loss=0.13581451934587135, metrics={'train_runtime': 7141.5035, 'train_samples_per_second': 4.812, 'train_steps_per_second': 0.602, 'total_flos': 1.7960169111552e+16, 'train_loss': 0.13581451934587135, 'epoch': 1.0})

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
input_text = "Americans"
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

In [ ]:
# Generate encoded text
with torch.no_grad():
  outputs = model.generate(input_ids, min_length=100, max_length=500, num_return_sequences=1, do_sample=True, temperature=0.7, top_k=50)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# outputs.shape

torch.Size([1, 104])

In [ ]:
print(generated_text)

Americans Americans want the government to do what it should do. And they want it to succeed. And they want it to work. And we want it to work. And we want it to work. And we want it to work together. And we want it to work in the right direction at the right time. So that's the way we’re going to work together. We’re going to work together. We’re going to work together. And we’re going to work together.


In [ ]:
model_path = './president-gpt2'
model.save_pretrained(model_path)

tokenizer_path = './president-gpt2-tok'
tokenizer.save_pretrained(tokenizer_path)

('./president-gpt2-tok/tokenizer_config.json',
 './president-gpt2-tok/special_tokens_map.json',
 './president-gpt2-tok/vocab.json',
 './president-gpt2-tok/merges.txt',
 './president-gpt2-tok/added_tokens.json',
 './president-gpt2-tok/tokenizer.json')

In [ ]:
!ls -l

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 16
drwxr-xr-x  2 root root 4096 Jul  4 11:06 president-gpt2
drwxr-xr-x  2 root root 4096 Jul  4 11:06 president-gpt2-tok
drwxr-xr-x 12 root root 4096 Jul  4 11:06 results
drwxr-xr-x  3 root root 4096 Jul  4 09:06 wandb
